In [ ]:
import requests as r
import pandas as pd
from decimal import Decimal
import pandas_gbq

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import pandas_utils as p
sys.path.pop()

import time
import os
import dotenv
from google.cloud import bigquery
from google.oauth2 import service_account

dotenv.load_dotenv()

# Set the environment variable to the path of your service account key file
# if os.environ["IS_RUNNING_LOCAL"]:
#         os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("PATH_TO_BQ_CREDS")
# else:
#         os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("BQ_APPLICATION_CREDENTIALS")

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("BQ_APPLICATION_CREDENTIALS")

# pandas_gbq.context.credentials = service_account.Credentials.from_service_account_file()

pandas_gbq.context.project = "oplabs-tools-data"
# Initialize a BigQuery client
# client = bigquery.Client()

In [ ]:
chain_mappings = {
        'zora': 'Zora',
        'pgn': 'Public Goods Network',
        'base': 'Base',
        'mode': 'Mode',
        'metal': 'Metal',
        'frax': 'Fraxtal'
        # Add more mappings as needed
    }

In [ ]:
sql_directory = "inputs/sql/"

query_names = [
        # Must match the file name in inputs/sql
        "frax_allltime_chain_activity"
]

In [ ]:
def convert_decimal_to_decimal_object(x):
    if isinstance(x, Decimal):
        return x
    else:
        return None

# Function to format values in the DataFrame
def format_value(x):
    if isinstance(x, Decimal):
        return str(x)
    else:
        return x

In [ ]:
for qn in query_names:
        # If we can do it programmatically from UI saved queries
        # query = client.get_job(query_name)
        # Read the SQL query from file
        with open(os.path.join(sql_directory, f"{qn}.sql"), "r") as file:
                query = file.read()
        print(qn)
        table_name = qn
        
        # Execute the query
        # query_job = client.query(query)
        df = pandas_gbq.read_gbq(query, credentials = os.getenv("BQ_APPLICATION_CREDENTIALS"))
        
        # Write to csv
        df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        # print(df.sample(5))
        time.sleep(1)
        
        # Write to Dune
        df['chain_raw'] = df['chain']
        df['chain'] = df['chain'].replace(chain_mappings)
        d.write_dune_api_from_pandas(df, table_name,table_description = table_name)
        # Print the results

In [ ]:
# Function to convert Decimal to float
def convert_decimal_to_decimal_object(x):
    if isinstance(x, Decimal):
        return x
    else:
        return None